In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import nltk
import pymorphy3
from tqdm.auto import tqdm
from collections import Counter
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec, FastText
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
# nltk.download('stopwords')
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings('ignore')

Конфигурационные переменные

In [3]:
DATASET_PATH = "../data/raw/labeled.csv"
SAVE_PROCESSED_PATH = "../data/processed/raw_features_extracted.csv"
GENERATED_PATH = "../data/processed/generated_agressive.csv"
PORNO_DATASET_PATH = "../data/raw/dataset_spot.xlsx"
SEED = 42
np.random.seed(SEED)

TOKEN_PATTERN = "[а-яёa-z]+"
SIGN_PATTERN = r'[!?*@#$&%^:;"\')(~`0-9]'
URL_PATTERN = r'https?://\S+|www\.\S+'
REPEATED_PATTERN = r'(.)\1{2,}'
PRONOUNS = ['я', 'ты', 'вы', 'он', 'она', 'оно', 'мы', 'они', 'вас', 'нас', 'их', 'его', 'её']


Считываем обновленный датасет

In [4]:
def read_dataset(data_path: str = SAVE_PROCESSED_PATH):
    dataset = pd.read_csv(data_path).drop(columns=['Unnamed: 0'])
    return dataset

comments_dataset = read_dataset()
comments_dataset.head()

,comment,toxic,text_length,potential_agressive_symbols,num_words,link_count,repeated_chars_count,pronoun_count,avg_word_length,caps_symbols
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0,35,1,5,0,1,0,6.000000,2
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0,121,0,21,0,0,1,4.761905,2
2,Собаке - собачья смерть\n,1.0,24,0,4,0,0,0,5.000000,1
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0,184,1,32,0,0,2,4.750000,3
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0,120,5,18,0,0,0,5.666667,4


Отсавим от него только две колонки

In [5]:
comments_dataset = comments_dataset.iloc[:, [0, 1]]
comments_dataset.head()

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0


Добавим к датасету несколько сэмплов из нового датасета на тему 18+

In [7]:
porno_dataset = pd.read_excel(PORNO_DATASET_PATH).iloc[:, [2, 3]]
porno_dataset.head()

,title,label
0,экс министр экономика молдова глава мидэи цель...,0
1,песня стать известный многий телезритель благо...,0
2,банши сезон серия бремя красота смотреть онлайн,0
3,новомосковск сыграть следж хоккеист алексински...,0
4,салат корейский морковь копчёный курица кукуру...,0


In [8]:
agressive_dataset = pd.read_csv(GENERATED_PATH).drop(columns = ['Unnamed: 0'])
agressive_dataset.head()

,comment,toxic
0,Почему никто не может сделать нормальный детск...,1
1,"Ты хоть представляешь, как меня бесит эта твоя...",1
2,"Ты когда-нибудь задумывался, какой ужас творит...",1
3,"Почему каждый раз, когда мне нужно купить комм...",1
4,"Ты что, совсем охренел? Какого фаллос черта ты...",1


In [9]:
comments_dataset = pd.concat([comments_dataset, agressive_dataset], axis = 0)

In [10]:
comments_dataset.shape

(15104, 2)

Корпус текстов

In [11]:
corpus_texts = comments_dataset.comment.values
corpus_texts[:10]

array(['Верблюдов-то за что? Дебилы, бл...\n',
       'Хохлы, это отдушина затюканого россиянина, мол, вон, а у хохлов еще хуже. Если бы хохлов не было, кисель их бы придумал.\n',
       'Собаке - собачья смерть\n',
       'Страницу обнови, дебил. Это тоже не оскорбление, а доказанный факт - не-дебил про себя во множественном числе писать не будет. Или мы в тебя верим - это ты и твои воображаемые друзья?\n',
       'тебя не убедил 6-страничный пдф в том, что Скрипалей отравила Россия? Анализировать и думать пытаешься? Ватник что ли?)\n',
       'Для каких стан является эталоном современная система здравоохранения РФ? Для Зимбабве? Ты тупой? хохлы\n',
       'В шапке были ссылки на инфу по текущему фильму марвел. Эти ссылки были заменены на фразу Репортим брипидора, игнорируем его посты. Если этого недостаточно, чтобы понять, что модератор абсолютный неадекват, и его нужно лишить полномочий, тогда эта борда пробивает абсолютное дно по неадекватности.\n',
       'УПАД Т! ТАМ НЕЛЬЗЯ СТРОИ

In [12]:
def tokenize(text):
    return re.findall(TOKEN_PATTERN, text.lower())

documents = [tokenize(str(text)) for text in corpus_texts]

In [13]:
occurence = Counter([text for document in documents for text in document])
len(occurence)

71861

In [14]:
stopword_set = set(nltk.corpus.stopwords.words('russian'))
stopword_set_english = set(nltk.corpus.stopwords.words('english'))
# stopword_set = stopword_set.union({'это', 'который', 'весь', 'наш', 'свой', 'ещё', 'её', 'ваш', 'также', 'итак'})
stopword_set = stopword_set.union(stopword_set_english)

In [16]:
corpus_texts

array(['Верблюдов-то за что? Дебилы, бл...\n',
       'Хохлы, это отдушина затюканого россиянина, мол, вон, а у хохлов еще хуже. Если бы хохлов не было, кисель их бы придумал.\n',
       'Собаке - собачья смерть\n', ...,
       'Кожа и волосы — это вечная проблема, передрочить которая преследует меня каждый день. Я ненавижу, когда люди допиздяшиваться начинают рассказывать о своих идеальных хуиный волосах и коже, будто это главное достижение в жизни. Эти бесконечные советы о том, как правильно ухаживать за собой, как использовать очередной волшебный крем или шампунь, просто выводят меня из себя. Как будто у всех сифилюга есть время и деньги на эти глупости! Я презираю все эти рекламные ролики и блогеров, которые пытаются внушить нам, что без их советов и продуктов мы обречены на вечные прыщи и тусклые волосы. Они просто наживаются на наших комплексах и примудохивать неуверенности. Это отвратительно! Почему никто не говорит о том, что отхуяшиться настоящая красота — это здоровье и внутр

In [ ]:
pip install natasha

In [51]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)


segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)

text = 'Посол Израиля на Украине Йоэль Лион признался, что пришел в шок, узнав о решении властей Львовской области объявить 2019 год годом лидера запрещенной в России Организации украинских националистов (ОУН) Степана Бандеры. Свое заявление он разместил в Twitter. «Я не могу понять, как прославление тех, кто непосредственно принимал участие в ужасных антисемитских преступлениях, помогает бороться с антисемитизмом и ксенофобией. Украина не должна забывать о преступлениях, совершенных против украинских евреев, и никоим образом не отмечать их через почитание их исполнителей», — написал дипломат. 11 декабря Львовский областной совет принял решение провозгласить 2019 год в регионе годом Степана Бандеры в связи с празднованием 110-летия со дня рождения лидера ОУН (Бандера родился 1 января 1909 года). В июле аналогичное решение принял Житомирский областной совет. В начале месяца с предложением к президенту страны Петру Порошенко вернуть Бандере звание Героя Украины обратились депутаты Верховной Рады. Парламентарии уверены, что признание Бандеры национальным героем поможет в борьбе с подрывной деятельностью против Украины в информационном поле, а также остановит «распространение мифов, созданных российской пропагандой». Степан Бандера (1909-1959) был одним из лидеров Организации украинских националистов, выступающей за создание независимого государства на территориях с украиноязычным населением. В 2010 году в период президентства Виктора Ющенко Бандера был посмертно признан Героем Украины, однако впоследствии это решение было отменено судом. '
doc = Doc(text)

print(doc.tokens)

doc = Doc(corpus_texts[0])

None


Пробуем сделать лемматизацию

In [52]:
lemmatizer = pymorphy3.MorphAnalyzer()

lemmatizer_cache = {}

def lemmatize(token):
    if lemmatizer.word_is_known(token):
        if token not in lemmatizer_cache:
            lemmatizer_cache[token] = lemmatizer.parse(token)[0].normal_form
        return lemmatizer_cache[token]
    return token

lemmatized_docs = [[lemmatize(token) for token in text] for text in tqdm(documents)]

cleared_docs = [[token for token in text if token not in stopword_set] for text in lemmatized_docs]

  0%|          | 0/15104 [00:00<?, ?it/s]

In [53]:
comments_dataset['cleared_text'] = cleared_docs

In [54]:
def len_zero(x):
    return len(x) != 0

comments_dataset['length_zero'] = comments_dataset[['cleared_text']].applymap(len_zero)
comments_dataset = comments_dataset.drop(columns = ['length_zero'])

In [55]:
def text_regenerator(x):
    return ' '.join(x)

comments_dataset['cleared_text'] = comments_dataset[['cleared_text']].applymap(text_regenerator)

In [56]:
comments_dataset.head()

,comment,toxic,cleared_text
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0,верблюд дебил бл
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0,хохол это отдушина затюканого россиянин мол во...
2,Собаке - собачья смерть\n,1.0,собака собачий смерть
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0,страница обновить дебил это оскорбление доказа...
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0,убедить страничный пдф скрипалей отравить росс...


Новые эмбеддинги

In [57]:
from navec import Navec

path = '/Users/a.chervonikov/Desktop/Purify/Purify/purify_ml/notebooks/navec_hudlit_v1_12B_500K_300d_100q.tar'
navec = Navec.load(path)

sentences = comments_dataset['cleared_text']

X_vector_fast_text = np.array([np.mean([navec[word] for word in sentence if navec.get(word) is not None] or [np.zeros(300)], axis=0) for sentence in sentences])
y = comments_dataset['toxic'].values

In [58]:
X_vector_fast_text.shape

(15104, 300)

In [66]:
X, y = comments_dataset['cleared_text'], comments_dataset['toxic']
X_train, X_test, y_train, y_test = train_test_split(X_vector_fast_text, 
                                                    y, 
                                                    test_size=0.2, 
                                                    stratify=y,
                                                    random_state=42)

In [67]:
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

LogisticRegression()

In [68]:
y_proba = log_reg.predict_proba(X_test)
positive_proba = y_proba[:, 1]
threshold = 0.5

y_pred_custom = (positive_proba > threshold).astype(int)


print(accuracy_score(y_true=y_test, y_pred=y_pred_custom), 
      accuracy_score(y_true=y_train, y_pred=log_reg.predict(X_train)))

0.6875206885137372 0.6881569146735083
